# BlockCluster EDA | Kyle McLester
The data for this project is found on [Kaggle](https://www.kaggle.com/rounakbanik/the-movies-dataset). Our goal for this portion of the project is to perform a basic exploratory analysis of the data and report our findings

## Importing Dependencies

In [261]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import operator
pd.options.mode.chained_assignment = None

## Loading Data

In [2]:
links = pd.read_csv('data/movielens/100k/links.csv', sep=',', encoding='latin-1')
movies = pd.read_csv('data/movielens/100k/movies.csv', sep=',', encoding='latin-1')
ratings = pd.read_csv('data/movielens/100k/ratings.csv', sep=',', encoding='latin-1', usecols=['userId','movieId','rating'])
tags = pd.read_csv('data/movielens/100k/tags.csv', sep=',', encoding='latin-1', usecols=['userId','movieId','tag'])

## Exploring Data

In [3]:
print(5*'*'+' COLUMN LABELS '+5*'*')
print('links: {}'.format(list(links.columns)))
print('movies: {}'.format(list(movies.columns)))
print('ratings: {}'.format(list(ratings.columns)))
print('tags: {}'.format(list(tags.columns)))

***** COLUMN LABELS *****
links: ['movieId', 'imdbId', 'tmdbId']
movies: ['movieId', 'title', 'genres']
ratings: ['userId', 'movieId', 'rating']
tags: ['userId', 'movieId', 'tag']


In [4]:
print(5*'*'+' DATA SHAPE '+5*'*')
print('links: {}'.format(links.shape))
print('movies: {}'.format(movies.shape))
print('ratings: {}'.format(ratings.shape))
print('tags: {}'.format(tags.shape))

***** DATA SHAPE *****
links: (9742, 3)
movies: (9742, 3)
ratings: (100836, 3)
tags: (3683, 3)


In [5]:
print(5*'*'+' RATING SUMMARY STATS '+5*'*')
ratings['rating'].describe()

***** RATING SUMMARY STATS *****


count    100836.000000
mean          3.501557
std           1.042529
min           0.500000
25%           3.000000
50%           3.500000
75%           4.000000
max           5.000000
Name: rating, dtype: float64

In [6]:
print(5*'*'+' NULL VALUE COUNTS '+5*'*')
print('links\n{}\n'.format(links.isnull().sum()))
print('movies\n{}\n'.format(movies.isnull().sum()))
print('ratings\n{}\n'.format(ratings.isnull().sum()))
print('tags\n{}\n'.format(tags.isnull().sum()))

***** NULL VALUE COUNTS *****
links
movieId    0
imdbId     0
tmdbId     8
dtype: int64

movies
movieId    0
title      0
genres     0
dtype: int64

ratings
userId     0
movieId    0
rating     0
dtype: int64

tags
userId     0
movieId    0
tag        0
dtype: int64



In [7]:
print(5*'*'+' REMOVE ROWS W/NULL VALUES ' + 5*'*')
links = links.dropna()
links.isnull().sum()

***** REMOVE ROWS W/NULL VALUES *****


movieId    0
imdbId     0
tmdbId     0
dtype: int64

In [8]:
print(5*'*'+' FIRST 5 ROWS '+5*'*')
print('links\n{}\n'.format(links.head()))
print('movies\n{}\n'.format(movies.head()))
print('ratings\n{}\n'.format(ratings.head()))
print('tags\n{}\n'.format(tags.head()))

***** FIRST 5 ROWS *****
links
   movieId  imdbId   tmdbId
0        1  114709    862.0
1        2  113497   8844.0
2        3  113228  15602.0
3        4  114885  31357.0
4        5  113041  11862.0

movies
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  

ratings
   userId  movieId  rating
0       1        1     4.0
1       1        3     4.0
2       1        6     4.0
3       1       47     5.0
4       1       50     5.0

tags
   userId  movieId   

In [9]:
# break up movies['genre'] into a string
movies['genres'] = movies['genres'].str.split('|')
movies['genres'] = movies['genres'].fillna('').astype('str')

In [10]:
movies.head()

movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                              genres  
0  ['Adventure', 'Animation', 'Children', 'Comedy...  
1               ['Adventure', 'Children', 'Fantasy']  
2                              ['Comedy', 'Romance']  
3                     ['Comedy', 'Drama', 'Romance']  
4                                         ['Comedy']

In [11]:
# extract year from title column
movies['year'] = [re.search('(\d){4}', x) for x in movies['title']]
movies = movies.dropna()
movies['year'] = [re.search('(\d){4}', x).group() for x in movies['title']]
movies = movies.replace(to_replace='\((.*)\)', value='', regex=True)
movies = movies.replace(to_replace=' +$', value='', regex=True)
movies['year'] = movies['year'].astype('int')

In [12]:
movies.head()

movieId                        title  \
0        1                    Toy Story   
1        2                      Jumanji   
2        3             Grumpier Old Men   
3        4            Waiting to Exhale   
4        5  Father of the Bride Part II   

                                              genres  year  
0  ['Adventure', 'Animation', 'Children', 'Comedy...  1995  
1               ['Adventure', 'Children', 'Fantasy']  1995  
2                              ['Comedy', 'Romance']  1995  
3                     ['Comedy', 'Drama', 'Romance']  1995  
4                                         ['Comedy']  1995

In [13]:
# remove title that were missing a year or are recorded after this study
movies = movies[movies['year']<=2018]

## Recommendation based on Genre

In [14]:
tf = TfidfVectorizer(analyzer='word',
                     ngram_range=(1,2),
                     min_df=0,
                     stop_words='english')

In [15]:
tfidf_matrix = tf.fit_transform(movies['genres'])
print('tfidf_matrix\nshape: {}'.format(tfidf_matrix.shape))

tfidf_matrix
shape: (9724, 174)


In [16]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cosine_sim[:4, :4]

array([[1.        , 0.31378594, 0.06107542, 0.05268521],
       [0.31378594, 1.        , 0.        , 0.        ],
       [0.06107542, 0.        , 1.        , 0.3516825 ],
       [0.05268521, 0.        , 0.3516825 , 1.        ]])

In [17]:
titles = movies['title']
indices = pd.Series(movies.index, index=movies['title'])

In [18]:
def genre_recommendations(title, num_titles):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:num_titles+1]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [19]:
genre_recommendations('Tomb Raider', 10)

1516               Flight of the Navigator
4274    Journey to the Center of the Earth
5004                             Explorers
5396                            D.A.R.Y.L.
6436                       Last Mimzy, The
6862                         City of Ember
9016        Doctor Who: The Waters of Mars
1506                       Black Hole, The
1554            Return from Witch Mountain
2803                            Titan A.E.
Name: title, dtype: object

In [21]:
genre_recommendations('Ant-Man and the Wasp', 10)

2861                                        Space Cowboys
3069                                           Innerspace
3927                        Adventures of Pluto Nash, The
5259                                Last Starfighter, The
6754    Indiana Jones and the Kingdom of the Crystal S...
7052                                     Land of the Lost
8691                                             Deadpool
9731                                              Gintama
560                                            Barbarella
969                                    Back to the Future
Name: title, dtype: object

## Recommendation based on Title

In [22]:
tf = TfidfVectorizer(analyzer='word',
                     ngram_range=(1, 2),
                     min_df=0,
                     stop_words='english')

In [23]:
tfidf_matrix =tf.fit_transform(movies['title'])
tfidf_matrix.shape

(9724, 15855)

In [24]:
%%time
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cosine_sim[:4, :4]

Wall time: 72.1 ms


array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [25]:
titles = movies['title']
indices = pd.Series(movies.index, index=movies['title'])

In [26]:
def title_recommendations(title, num_titles):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:num_titles+1]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [27]:
title_recommendations('Tomb Raider', 10)

9433                         Rogue One: A Star Wars Story
567                                                  Solo
6823                            Star Wars: The Clone Wars
9645                             Star Wars: The Last Jedi
8908                        The Star Wars Holiday Special
7367    Empire of Dreams: The Story of the 'Star Wars'...
1979            Star Wars: Episode I - The Phantom Menace
898        Star Wars: Episode V - The Empire Strikes Back
1570                                           L.A. Story
2227                                     Story of Us, The
Name: title, dtype: object

In [28]:
title_recommendations('Toy Story', 10)

2355           Toy Story 2
7355           Toy Story 3
3595              Toy, The
1570            L.A. Story
2227      Story of Us, The
4156       Story of O, The
4089          Toy Soldiers
3187            Love Story
2110    Christmas Story, A
4047           Ghost Story
Name: title, dtype: object

## Average Rating Given by Each User

In [64]:
merged_df = ratings.join(movies.set_index('movieId'), on='movieId')

In [65]:
merged_df[merged_df['userId']==1]

userId  movieId  rating                    title  \
0         1        1     4.0                Toy Story   
1         1        3     4.0         Grumpier Old Men   
2         1        6     4.0                     Heat   
3         1       47     5.0                    Seven   
4         1       50     5.0      Usual Suspects, The   
..      ...      ...     ...                      ...   
227       1     3744     4.0                    Shaft   
228       1     3793     5.0                    X-Men   
229       1     3809     4.0          What About Bob?   
230       1     4006     4.0  Transformers: The Movie   
231       1     5060     5.0                  M*A*S*H   

                                                genres    year  
0    ['Adventure', 'Animation', 'Children', 'Comedy...  1995.0  
1                                ['Comedy', 'Romance']  1995.0  
2                      ['Action', 'Crime', 'Thriller']  1995.0  
3                              ['Mystery', 'Thriller']  1995.0  
4                     ['Crime', 'Mystery', 'Thriller']  1995.0  
..                                                 ...     ...  
227                    ['Action', 'Crime', 'Thriller']  2000.0  
228                  ['Action', 'Adventure', 'Sci-Fi']  2000.0  
229                                         ['Comedy']  1991.0  
230   ['Adventure', 'Animation', 'Children', 'Sci-Fi']  1986.0  
231                         ['Comedy', 'Drama', 'War']  1970.0  

[232 rows x 6 columns]

In [66]:
user_avg_ratings = pd.DataFrame(columns=['userId','avg_rating'])
userId = []
avg_rating = []
for user in merged_df['userId'].unique():
    userId.append(user)
    avg_rating.append(merged_df[merged_df['userId']==user]['rating'].sum()/len(merged_df[merged_df['userId']==user]))
user_avg_ratings['userId'] = userId
user_avg_ratings['avg_rating'] = avg_rating

In [67]:
user_avg_ratings = user_avg_ratings.set_index('userId')

In [68]:
user_avg_ratings.describe()

avg_rating
count  610.000000
mean     3.657222
std      0.480635
min      1.275000
25%      3.360000
50%      3.694385
75%      3.997500
max      5.000000

## Average Rating for Each Movie (Naive)

In [77]:
avg_movie_ratings = pd.DataFrame(columns=['movieId','avg_rating'])
movieId = []
avg_rating = []
for movie in merged_df['movieId'].unique():
    movieId.append(movie)
    avg_rating.append(merged_df[merged_df['movieId']==movie]['rating'].sum()/len(merged_df[merged_df['movieId']==movie]))
avg_movie_ratings['movieId'] = movieId
avg_movie_ratings['avg_rating'] = avg_rating

In [112]:
avg_movie_ratings = avg_movie_ratings.join(movies.set_index('movieId'), on='movieId')
avg_movie_ratings = avg_movie_ratings.drop(['genres','year'], axis=1)
avg_movie_ratings = avg_movie_ratings.dropna()

In [127]:
# 10 lowest rated movies
avg_movie_ratings.sort_values(by=['avg_rating'])[:10]

movieId  avg_rating                                title
8941     7114         0.5                       Collector, The
8859   122246         0.5                        Tooth Fairy 2
8531     8632         0.5                       Secret Society
7822    82095         0.5                              Skyline
9298   136297         0.5    Mortal Kombat: The Journey Begins
6513    54934         0.5                Brothers Solomon, The
6518    67799         0.5  The Butterfly Effect 3: Revelations
9610    61818         0.5             Crow, The: Wicked Prayer
6535   134528         0.5                                Aloha
6539   138798         0.5          Joe Dirt 2: Beautiful Loser

In [129]:
# 10 highest rated movies
avg_movie_ratings.sort_values(by=['avg_rating'], ascending=False)[:10]

movieId  avg_rating                                     title
4275     4788         5.0          Moscow Does Not Believe in Tears
8685   120130         5.0       Into the Forest of Fireflies' Light
7102     6021         5.0                      American Friend, The
5572     5607         5.0                          Son of the Bride
4277     5888         5.0                                   Brother
8557    71268         5.0  Tyler Perry's I Can Do Bad All by Myself
8558    93320         5.0                         Trailer Park Boys
4278     5889         5.0                          Cruel Romance, A
8580      467         5.0                           Live Nude Girls
8581      876         5.0                                Supercop 2

In [131]:
avg_movie_ratings['avg_rating'].describe()

count    9706.000000
mean        3.261957
std         0.869360
min         0.500000
25%         2.800000
50%         3.416667
75%         3.909773
max         5.000000
Name: avg_rating, dtype: float64

## Creating User Profiles

In [193]:
profiles = {}

for user in merged_df['userId'].unique():
    username = 'user_' + str(user)
    
    temp = merged_df[merged_df['userId']==user]
    
    titles = temp['title']
    ratings = temp['rating']
    ratings = list(ratings)
    
    movie_list = {}
    for i, title in enumerate(titles):
        movie_list[title] = ratings[i]
        
    profiles[username] = movie_list

In [199]:
def get_common_movies(user_A, user_B):
    return [movie for movie in profiles[user_A] if movie in profiles[user_B]]

In [212]:
# number of available user profiles
len(profiles)

610

In [213]:
get_common_movies('user_234','user_610')

['Toy Story',
 'Batman Forever',
 'Dumb & Dumber',
 'Star Wars: Episode IV - A New Hope',
 'Ace Ventura: Pet Detective',
 'Speed',
 'Jurassic Park',
 'Batman',
 'Wizard of Oz, The',
 'Die Hard',
 'E.T. the Extra-Terrestrial',
 'Star Wars: Episode V - The Empire Strikes Back',
 'Raiders of the Lost Ark',
 'Aliens',
 'Star Wars: Episode VI - Return of the Jedi',
 'Alien',
 'Army of Darkness',
 'Indiana Jones and the Last Crusade',
 'Face/Off',
 'As Good as It Gets',
 'Labyrinth',
 'Lethal Weapon',
 'Saving Private Ryan',
 'Dark Crystal, The',
 'NeverEnding Story, The',
 "Bug's Life, A",
 'Matrix, The',
 'Star Wars: Episode I - The Phantom Menace',
 'RoboCop',
 'Toy Story 2',
 'Beach, The',
 'Teenage Mutant Ninja Turtles',
 'Flintstones in Viva Rock Vegas, The',
 'Shanghai Noon',
 'Starman',
 'Chicken Run',
 'X-Men',
 "Charlie's Angels",
 'Cast Away',
 'Memento',
 'Spy Kids',
 'Mummy Returns, The',
 'Shrek',
 'Jurassic Park III']

In [234]:
# get reviews from common movies
def get_reviews(user_A, user_B):
    common_movies = get_common_movies(user_A, user_B)
    return [(profiles[user_A][movie], profiles[user_B][movie]) for movie in common_movies]

In [235]:
get_reviews('user_70','user_391')

[(4.0, 4.0),
 (4.5, 5.0),
 (4.0, 5.0),
 (5.0, 5.0),
 (5.0, 5.0),
 (5.0, 5.0),
 (4.5, 5.0),
 (5.0, 5.0),
 (5.0, 5.0),
 (5.0, 4.0),
 (5.0, 4.0),
 (4.5, 4.0),
 (4.0, 4.0),
 (3.5, 4.0),
 (4.5, 3.0),
 (4.0, 4.0),
 (4.0, 4.0),
 (3.5, 4.0),
 (4.0, 4.0),
 (4.5, 4.0)]

In [236]:
# get reviews from common movies
def get_reviews_detailed(user_A, user_B):
    common_movies = get_common_movies(user_A, user_B)
    reviews = [(profiles[user_A][movie], profiles[user_B][movie]) for movie in common_movies]
    result = zip(common_movies, reviews)
    return set(result)

In [237]:
get_reviews_detailed('user_70','user_391')

{('American Beauty', (4.0, 4.0)),
 ('Apollo 13', (4.0, 4.0)),
 ('Back to the Future', (4.0, 4.0)),
 ('Beautiful Mind, A', (4.5, 4.0)),
 ('Breakfast Club, The', (4.0, 4.0)),
 ('Cinema Paradiso', (5.0, 5.0)),
 ('Dead Poets Society', (5.0, 4.0)),
 ('E.T. the Extra-Terrestrial', (4.5, 5.0)),
 ("Ferris Bueller's Day Off", (3.5, 4.0)),
 ('Forrest Gump', (4.0, 5.0)),
 ('Godfather, The', (5.0, 5.0)),
 ('Good Will Hunting', (4.5, 3.0)),
 ("It's a Wonderful Life", (5.0, 5.0)),
 ("One Flew Over the Cuckoo's Nest", (5.0, 5.0)),
 ("Schindler's List", (5.0, 5.0)),
 ('Shawshank Redemption, The', (4.5, 5.0)),
 ('Shrek', (4.0, 4.0)),
 ('Stand by Me', (4.5, 4.0)),
 ('Star Wars: Episode VI - Return of the Jedi', (5.0, 4.0)),
 ('Truman Show, The', (3.5, 4.0))}

In [238]:
def euclidean_distance(points):
    squared_diff = [(point[0] - point[1]) ** 2 for point in points]
    summed_squared_diffs = sum(squared_diff)
    distance = np.sqrt(summed_squared_diffs)
    return distance

In [239]:
def similarity(reviews):
    return 1/(1+euclidean_distance(reviews))

In [240]:
def get_user_similarity(user_A, user_B):
    reviews = get_reviews(user_A, user_B)
    return similarity(reviews)

In [246]:
get_user_similarity('user_162','user_197')

0.20521309615767264

In [275]:
def recommend_movies(user, num_suggestions):
    similarity_scores = [(get_user_similarity(user, other), other) for other in profiles if other != user]
    # get sim scores for all users
    similarity_scores.sort()
    similarity_scores.reverse()
    # similarity_scores = similarity_scores[0:num_suggestions]
    
    recommendations = {}
    for similarity, other in similarity_scores:
        reviewed = profiles[other]
        for movie in reviewed:
            if movie not in profiles[user]:
                weight = similarity * reviewed[movie]
                if movie in recommendations:
                    sim, weights = recommendations[movie]
                    recommendations[movie] = (sim + similarity, weights + [weight])
                else:
                    recommendations[movie] = (similarity, [weight])
                    
    for recommendation in recommendations:
        similarity, movie = recommendations[recommendation]
        recommendations[recommendation] = sum(movie) / similarity
        
    sorted_recommendations = sorted(recommendations.items(), key=operator.itemgetter(1), reverse=True)
    return sorted_recommendations[:num_suggestions]

In [278]:
recommend_movies('user_203', 10)

[('Jonah Who Will Be 25 in the Year 2000', 5.000000000000001),
 ('Come and See', 5.000000000000001),
 ('Story of Women', 5.0),
 ('Passenger, The', 5.0),
 ('Marriage of Maria Braun, The', 5.0),
 ('61*', 5.0),
 ('Thin Line Between Love and Hate, A', 5.0),
 ('Animals are Beautiful People', 5.0),
 ('Woman Is a Woman, A', 5.0),
 ('Thousand Clowns, A', 5.0)]

In [279]:
recommend_movies('user_120', 10)

[('Particle Fever', 5.000000000000001),
 ('Heidi Fleiss: Hollywood Madam', 5.000000000000001),
 ('Lamerica', 5.0),
 ('Entertaining Angels: The Dorothy Day Story', 5.0),
 ('Story of Women', 5.0),
 ('Passenger, The', 5.0),
 ('Marriage of Maria Braun, The', 5.0),
 ('One I Love, The', 5.0),
 ('Laggies', 5.0),
 ('Delirium', 5.0)]